# Lesson 13 - Advanced SQL Joins and Performance Tuning


## `FULL OUTER JOIN` OR `FULL JOIN` 

<img src="../SQL/ERD DAND.jpg" width="600" height="400">

Say you're an analyst at Parch & Posey and you want to see:

- each account who has a sales rep and each sales rep that has an account (all of the columns in these returned rows will be full)
- but also each account that does not have a sales rep and each sales rep that does not have an account (some of the columns in these returned rows will be empty)

This type of question is rare, but `FULL OUTER JOIN` is perfect for it. In the following SQL Explorer, write a query with `FULL OUTER JOIN` to fit the above described Parch & Posey scenario (selecting all of the columns in both of the relevant tables, accounts and sales_reps) then answer the subsequent multiple choice quiz.

`SELECT *
FROM accounts
FULL JOIN sales_reps ON accounts.sales_rep_id = sales_reps.id
WHERE accounts.sales_rep_id IS NULL OR sales_reps.id IS NULL`


## Inequality JOINs

If you recall from earlier lessons on joins, the join clause is evaluated before the where clause -- filtering in the join clause will eliminate rows before they are joined, while filtering in the `WHERE` clause will leave those rows in and produce some nulls.

**Example: In the following SQL Explorer, write a query that left joins the accounts table and the sales_reps tables on each sale rep's ID number and joins it using the < comparison operator on accounts.primary_poc and sales_reps.name, like so: `accounts.primary_poc < sales_reps.name`. The query results should be a table with three columns: the account name (e.g. Johnson Controls), the primary contact name (e.g. Cammy Sosnowski), and the sales representative's name (e.g. Samuel Racine). Then answer the subsequent multiple choice question.**

`SELECT a.name account, a.primary_poc, s.name sales_rep
FROM accounts a
LEFT JOIN sales_reps s
ON a.sales_rep_id = s.id
AND a.primary_poc < s.name`

Note that the primary point of contact's full name comes *before* the sales representative's name alphabetically.

More information on string comparisons in SQL: https://stackoverflow.com/questions/26080187/sql-string-comparison-greater-than-and-less-than-operators/26080240#26080240



## `SELF JOINs`

One of the most common use cases for self JOINs is in cases where two events occurred, one after another. As you may have noticed in the previous video, using inequalities in conjunction with self JOINs is common.

**Example: A query of web orders being processed within one day of another with the same account_id.**

`SELECT we1.id AS we_id,
       we1.account_id AS we1_account_id,
       we1.occurred_at AS we1_occurred_at,
       we1.channel AS we1_channel,
       we2.id AS we2_id,
       we2.account_id AS we2_account_id,
       we2.occurred_at AS we2_occurred_at,
       we2.channel AS we2_channel
  FROM web_events we1 
 LEFT JOIN web_events we2
   ON we1.account_id = we2.account_id
  AND we1.occurred_at > we2.occurred_at
  AND we1.occurred_at <= we2.occurred_at + INTERVAL '1 day'
ORDER BY we1.account_id, we2.occurred_at`

Information on how to use `INTERVAL` is here: https://www.postgresql.org/docs/8.2/static/functions-datetime.html



## Appending Data via `UNION` and `UNION ALL`

This combines two tables and is much like a Venn diagram. For `UNION`, duplicate matches are dropped so only distinct values are retained. It involves running two SELECT statements with the `UNION` statement. Note that if you only run `UNION` it will drop duplicates whereas `UNION ALL` will include duplicates. 

SQLs two strict rules for appending data:
    1. Both tables must have the same number of columns.
    2. Those columns must have the same data types in the same order as the first table.
    
Note that column names don't have to be the same name but often are.  Note that you can pretreat `UNION ALL` with `WHERE`. Also you'll often use `UNION ALL` more frequently than `UNION`.

**Example: Appending Data via UNION**

`SELECT *
FROM accounts
UNION ALL
SELECT *
FROM accounts`

**Example: Pretreating Tables before doing a UNION**

`SELECT *
FROM accounts
WHERE name = 'Walmart'
UNION
SELECT *
FROM accounts
WHERE name = 'Disney'`


**Example: Performing Operations on a Combined Dataset**

Perform the union in your first query (under the Appending Data via UNION header) in a common table expression and name it double_accounts. Then do a COUNT the number of times a name appears in the double_accounts table. If you do this correctly, your query results should have a count of 2 for each name.

`WITH double_accounts AS (
    SELECT *
      FROM accounts
    UNION ALL
    SELECT *
      FROM accounts )
SELECT name,
 COUNT(*) AS name_count
 FROM double_accounts 
GROUP BY 1
ORDER BY 2 DESC`



## Performance Tuning

**How You Can and Can't Control Performance**

One way to make a query run faster is to reduce the number of calculations that need to be performed. Some of the high-level things that will affect the number of calculations a given query will make include:

- Table size
- Joins
- Aggregations

Query runtime is also dependent on some things that you can’t really control related to the database itself:

- Other users running queries concurrently on the database
- Database software and optimization (e.g. Postgres is optimized differently than Redshift)

Tips to limit query run time:
- Put the `LIMIT` statement in a sub-query to limit data as it is executed first. Not the outer query otherwise it will still just compute the full query and then `LIMIT` the results in the last step. This is redundant.
- Limit your query to a small time window if you have time-series data.
- Test your query on a subset of data first.
- Reduce table sizes first before joining them using pre-aggregations or other filters with `WHERE`.
- You can use `EXPLAIN` at the start of your query to see which statements are executed first and how long they take. The modify the steps that are expensive. Then run `EXPLAIN` again to see if time has been reduced.
- `FULL JOIN` and `COUNT` run pretty fast. `COUNT(DISTINCT)` is slow.
